In [ ]:
from utils import *
from scipy import optimize
import matplotlib

In [ ]:
x = np.array([0.2672612, 0.5345225, 0.8017837]).reshape(1,-1)
y = np.array([0.4558423, 0.5698029, 0.6837635]).reshape(1,-1)
for i in range(0, 3):
    kernel = NTK(depth = 3+i, bias=3)
    # kernel = Matern(nu=0.5)
    
    print(f'Depth: {3+i} Value: ', kernel(x, y))

In [ ]:
z = np.linspace(-2, 2, 100)
x = (z**2 + 1) * np.sin(np.linspace(-np.pi, np.pi, 100))
y = (z**2 + 1) * np.cos(np.linspace(-np.pi, np.pi, 100))

datasets = processing(x, y, z, noise = 0.15)
X, y = datasets['orig']
X_norm, _ = datasets['norm']
X_train, y_train, y_train_noisy = datasets['orig train']
X_norm_train, _, _ = datasets['norm train']

data = (X_train, y_train, X, y)

In [ ]:
ntk_1 = (
    ConstantKernel(constant_value=1.0, constant_value_bounds=(1e-9, 1e5)) * 
    NTK(depth=3, c=2, bias=np.random.randn(50,50).ravel(), 
        bias_bounds=(1e-9, 1e2))
)

gp_ntk_1 = GPR(kernel=ntk_1, normalize_y=True, alpha=1e-5, n_restarts_optimizer=1, random_state=3480795)
gp_ntk_1.fit(X_train, y_train)

In [ ]:
fig, ax = plot((X, X_train), (y.ravel(), y_train_noisy), typ='data', title=r"$z = f(x, y)$", figsize=(8, 2.5))
fig.savefig('./illustrative/dataset.svg')

In [4]:
def g(ell, gp, data, residual, mean_ntk = None):
    try:
        gp.set_params(**{'kernel__k2__length_scale': ell})
    except:
        gp.set_params(**{'kernel__k1__k2__length_scale': ell})

    gp.fit(data[0], data[1])
    mean = gp.predict(data[2])

    if type(mean_ntk) == np.ndarray:
        return -np.sum((mean_ntk - mean)**2)
    else:
        return -np.corrcoef((residual)[:, 0], (data[3]-mean)[:, 0])[0, 1]        

In [ ]:
ntk_1 = (
    ConstantKernel(constant_value=1.0, constant_value_bounds=(1e-9, 1e5)) * 
    NTK(depth=3, c=2, bias=1e-9, 
        bias_bounds=(1e-9, 1e2))
)

gp_ntk_1 = GPR(kernel=ntk_1, normalize_y=True, alpha=1e-5, n_restarts_optimizer=9, random_state=3480795)
gp_ntk_1.fit(X_train, y_train)
mean_ntk_1 = gp_ntk_1.predict(X)

lpk_1 = (
    ConstantKernel(constant_value=gp_ntk_1.kernel_.get_params()['k1__constant_value'], constant_value_bounds='fixed') *#(1e-9, 1e5)) * 
    Matern(length_scale=1,
        length_scale_bounds='fixed',#(1e-9, 1e3), 
        nu=1/2)
)

gp_lpk_1 = GPR(kernel=lpk_1, normalize_y=True, alpha=1e-5, n_restarts_optimizer=9, random_state=3480795)
ell_lpk = optimize.minimize_scalar(g, args=(gp_lpk_1, data, y-mean_ntk_1), method='bounded', bounds=[0.0001, 1000], options={'disp': 3, 'maxiter': 10000})
gp_lpk_1.set_params(**{'kernel__k2__length_scale': ell_lpk.x})
gp_lpk_1.fit(X_train, y_train)
mean_lpk_1 = gp_lpk_1.predict(X)

# np.linalg.norm(xn-yn)/np.log(1/ntk_val_n)

# lpk_1 = (
#     ConstantKernel(constant_value=1.0, constant_value_bounds=(1e-9, 1e5)) * 
#     Matern(length_scale=np.linalg.norm(xn-yn)/np.log(1/ntk_val_n),
#         length_scale_bounds='fixed',#(1e-9, 1e3), 
#         nu=1/2)
# )



In [ ]:
plot(X, mean_ntk_1.ravel(), typ='data')

In [ ]:
plot(X, mean_lpk_1.ravel(), typ='data')

In [22]:
ntk_1 = (
    ConstantKernel(constant_value=1.0, constant_value_bounds=(1e-9, 1e5)) * 
    NTK(depth=3, c=2, bias=1e-9, 
        bias_bounds=(1e-9, 1e2))
)

gp_ntk_1 = GPR(kernel=ntk_1, normalize_y=True, alpha=1e-5, n_restarts_optimizer=9, random_state=3480795)
gp_ntk_1.fit(X_norm_train, y_train)
mean_ntk_1 = gp_ntk_1.predict(X_norm)
gp_ntk_1.kernel_

/anaconda/envs/thesis/lib/python3.9/site-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__bias is close to the specified lower bound 1e-09. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


1.05**2 * NTK(depth=3, c=2.000, bias=0.000)

In [58]:
X_norm.reshape(2,-1)[0].reshape(-1,1).shape

(100, 1)

In [62]:
# ntk = NTK(depth=3,c=2, bias=0.0)
# for i in range(0,100):
ele1 = 0 
ele2 = 4
x = normalize(np.random.randint(0, 10000000, 10).reshape(1,-1))
y = 
print(np.linalg.norm(X_norm_train.reshape(2,-1)[0].reshape(1,-1) - X_norm_train.reshape(2,-1)[1].reshape(1,-1))/np.log(1/gp_ntk_1.kernel_(X_norm_train.reshape(2,-1)[0].reshape(1,-1), X_norm_train.reshape(2,-1)[1].reshape(1,-1))))

[[-6.3226097]]


In [63]:
data = (X_norm_train, y_train, X_norm, y)
lpk_1 = (
    ConstantKernel(constant_value=gp_ntk_1.kernel_.get_params()['k1__constant_value'], constant_value_bounds='fixed') *#(1e-9, 1e5)) * 
    Matern(length_scale=1,
        length_scale_bounds='fixed',#(1e-9, 1e3), 
        nu=1/2)
)

gp_lpk_1 = GPR(kernel=lpk_1, normalize_y=True, alpha=1e-5, n_restarts_optimizer=9, random_state=3480795)
ell_lpk = optimize.minimize_scalar(g, args=(gp_lpk_1, data, y-mean_ntk_1), method='bounded', bounds=[0.0001, 1000], options={'disp': 3, 'maxiter': 10000})
gp_lpk_1.set_params(**{'kernel__k2__length_scale': ell_lpk.x})
gp_lpk_1.fit(X_train, y_train)
mean_lpk_1 = gp_lpk_1.predict(X)

gp_lpk_1.kernel_
# np.linalg.norm(xn-yn)/np.log(1/ntk_val_n)

# lpk_1 = (
#     ConstantKernel(constant_value=1.0, constant_value_bounds=(1e-9, 1e5)) * 
#     Matern(length_scale=np.linalg.norm(xn-yn)/np.log(1/ntk_val_n),
#         length_scale_bounds='fixed',#(1e-9, 1e3), 
#         nu=1/2)
# )


 
 Func-count     x          f(x)          Procedure
    1        381.966    -0.999563        initial
    2        618.034    -0.999228        golden
    3        236.068    -0.999704        golden
    4        145.898    -0.999763        golden
    5          90.17    -0.999787        golden
    6        55.7282    -0.999799        golden
    7         34.442    -0.999806        golden
    8        21.2863    -0.999812        golden
    9        13.1557     -0.99982        golden
   10        8.13072     -0.99983        golden
   11         5.0251    -0.999844        golden
   12        3.10572    -0.999858        golden
   13        1.91948     -0.99985        golden
   14        3.25184    -0.999857        parabolic
   15        2.84092    -0.999859        parabolic
   16        2.48896    -0.999859        golden
   17        2.55605     -0.99986        parabolic
   18        2.59536     -0.99986        parabolic
   19        2.58238     -0.99986        parabolic
   20        2.5829

1.05**2 * Matern(length_scale=2.58, nu=0.5)

In [64]:
gp_ntk_1.kernel_(X_train)

array([[17.10302687, 16.25170859, 16.74975671, ...,  9.52836302,
         6.22665586, 17.06257648],
       [16.25170859, 27.42264833, 24.01036965, ..., 17.67434745,
         5.14003575, 20.72893125],
       [16.74975671, 24.01036965, 24.05874126, ..., 14.977716  ,
         5.44969887, 21.03142776],
       ...,
       [ 9.52836302, 17.67434745, 14.977716  , ..., 18.3408313 ,
         2.95433661, 12.57242169],
       [ 6.22665586,  5.14003575,  5.44969887, ...,  2.95433661,
         6.95816731,  5.76644405],
       [17.06257648, 20.72893125, 21.03142776, ..., 12.57242169,
         5.76644405, 21.04016856]])

In [65]:
gp_lpk_1.kernel_(X_train)

array([[1.09690593, 0.46398951, 0.60314694, ..., 0.31468488, 0.37883852,
        0.77404307],
       [0.46398951, 1.09690593, 0.83951726, ..., 0.51781188, 0.17585816,
        0.6535764 ],
       [0.60314694, 0.83951726, 1.09690593, ..., 0.45425645, 0.2208676 ,
        0.85292237],
       ...,
       [0.31468488, 0.51781188, 0.45425645, ..., 1.09690593, 0.16863618,
        0.39212303],
       [0.37883852, 0.17585816, 0.2208676 , ..., 0.16863618, 1.09690593,
        0.27560615],
       [0.77404307, 0.6535764 , 0.85292237, ..., 0.39212303, 0.27560615,
        1.09690593]])